In [ ]:
import pandas as pd #principal herramienta de gestion de datasets como dataframe
import numpy as np #para trabajar con arrays y algunas cuestiones estadisticas
import matplotlib.pyplot as plt #herramienta grafica
from sklearn import preprocessing #procesamiento de detasets 
from sklearn.metrics import mean_squared_error #herramienta estadistica
import seaborn as sns #herramienta grafica
import warnings #evita que se llene de warnings el script
warnings.filterwarnings("ignore", category=FutureWarning) #evita que se llene de warnings el script
sns.set() 
from sklearn.model_selection import cross_validate #para validacion cruzada 
from sklearn.metrics import accuracy_score #metrica accuracy
from sklearn.metrics import precision_score #metrica precisión
from sklearn.metrics import recall_score #metrica recall
from sklearn.metrics import f1_score #metrica f1 score
from sklearn.model_selection import train_test_split #para separar el dataset en entrenamiento y testeo
from __future__  import absolute_import,division,print_function,unicode_literals #parte de la importación y persistencia de tensorflow en Colab
import tensorflow as tf #libreria para usar el modelo
from sklearn.preprocessing import OrdinalEncoder #preprocesamiento de datos 
from sklearn.preprocessing import LabelEncoder #preprocesamiento de datos

In [ ]:
#clono el repositorio

In [ ]:
!git clone https://github.com/soyHenry/Datathon.git

fatal: destination path 'Datathon' already exists and is not an empty directory.


In [ ]:
#ingesto los datos en dataframe de pandas

In [ ]:
df_train=pd.read_csv('Datathon/hospitalizaciones_train.csv')

In [ ]:
df1=df_train.copy() #realizo una copia para trabajar con back up de los datos crudos 

In [ ]:
#creo una funcion para adaptar la columna label a categórica

In [ ]:
def transf_label(df):
    lista_dias=df['Stay (in days)']
    lista_estancia=[]
    for i in lista_dias:
        if i <=8: 
            i=0
        elif i>8:
            i=1
        lista_estancia.append(i)
    array_estancia=np.array(lista_estancia)
    array_estancia.reshape(1,-1)
    array_estancia.shape
    df['estancia']=array_estancia
    df.drop(columns=['Stay (in days)'], inplace=True)
    return df

In [ ]:
transf_label(df1) #aplico la función

,Available Extra Rooms in Hospital,Department,Ward_Facility_Code,doctor_name,staff_available,patientid,Age,gender,Type of Admission,Severity of Illness,health_conditions,Visitors with Patient,Insurance,Admission_Deposit,estancia
0,4,gynecology,D,Dr Sophia,0,33070,41-50,Female,Trauma,Extreme,Diabetes,4,Yes,2966.408696,0
1,4,gynecology,B,Dr Sophia,2,34808,31-40,Female,Trauma,Minor,Heart disease,2,No,3554.835677,1
2,2,gynecology,B,Dr Sophia,8,44577,21-30,Female,Trauma,Extreme,Diabetes,2,Yes,5624.733654,0
3,4,gynecology,D,Dr Olivia,7,3695,31-40,Female,Urgent,Moderate,None,4,No,4814.149231,0
4,2,anesthesia,E,Dr Mark,10,108956,71-80,Male,Trauma,Moderate,Diabetes,2,No,5169.269637,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409995,1,gynecology,D,Dr Sarah,1,63105,31-40,Female,Trauma,Minor,Asthama,3,Yes,3902.291076,0
409996,5,gynecology,D,Dr Nathan,5,129292,21-30,Female,Trauma,Extreme,Other,4,No,4771.995223,1
409997,4,gynecology,D,Dr Sarah,9,11399,21-30,Female,Emergency,Minor,Asthama,4,Yes,3816.994210,1
409998,10,gynecology,D,Dr Olivia,10,75003,21-30,Female,Trauma,Moderate,Other,2,Yes,3841.577491,1


In [ ]:
#Realizo una transormación de las columnas categoricas a numéricas 
#con label encoder en las no ordinales y ordinal encoder en las que si

In [ ]:
le = preprocessing.LabelEncoder()
df1['Insurance'] = le.fit_transform(df1['Insurance'])
df1['health_conditions'] = le.fit_transform(df1['health_conditions'])
df1['Type of Admission'] = le.fit_transform(df1['Type of Admission'])
df1['gender'] = le.fit_transform(df1['gender'])
df1['doctor_name'] = le.fit_transform(df1['doctor_name'])
df1['Ward_Facility_Code'] = le.fit_transform(df1['Ward_Facility_Code'])
df1['Department'] = le.fit_transform(df1['Department'])

In [ ]:
enc = OrdinalEncoder()
df1['Age'] = enc.fit_transform(df1[['Age']])
df1['Severity of Illness'] = enc.fit_transform(df1[['Severity of Illness']])


In [ ]:
df2=df1.copy() #se generan 2 data frame con distintas columnas para aplicar el modelo y analizar diferencias 
df2=df2[['Department','Age', 'doctor_name','estancia']]
df1=df1[['Department','Age','estancia']]


In [ ]:
df1.to_csv('train_1',index=False) #exporto como csv para poder ingresarlo al modelo de keras

In [ ]:
df2.to_csv('train_2',index=False) #exporto como csv para poder ingresarlo al modelo de keras

In [ ]:
y1=df1[['estancia']]
y2=df2[['estancia']]
#TODO _ x=df[['Available Extra Rooms in Hospital','Department','Ward_Facility_Code','doctor_name','staff_available','patientid','Age','gender','Type of Admission','Severity of Illness','health_conditions','Visitors with Patient','Insurance','Admission_Deposit']]
X1=df1[['Department','Age']]
X2=df2[['Department','Age', 'doctor_name']]

In [ ]:
#realizo la primera partición de los datos, luego se hará una validación cruzada con el metodo Shuffle

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.30, random_state=20) 

# datos, dejo un 30% para testear

print(X1_train.shape, y1_train.shape, X1_test.shape, y1_test.shape)

(287000, 2) (287000, 1) (123000, 2) (123000, 1)


In [ ]:
#realizo la primera partición de los datos, luego se hará una validación cruzada

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.30, random_state=20) 

# datos, dejo un 20% para testear

print(X2_train.shape, y2_train.shape, X2_test.shape, y2_test.shape)

(287000, 3) (287000, 1) (123000, 3) (123000, 1)


In [ ]:
CSV_COLUMN_NAMES= ['Department','Age', 'doctor_name', 'estancia']
ESTANCIA = [1,0]
# Lets define some constants to help us later on

In [ ]:
train_path= tf.keras.utils.get_file('train_1.csv') # este paso esta en proceso aún 
test_path = tf.keras.utils.get_file("train_1.csv") # este paso esta en proceso aún 

train= pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
#Aquí uso keras (un módulo dentro de Tensorflow) para ingestar conjuntos de datos y leerlos con pandas

In [ ]:
def input_fn(features,labels,training=True,batch_size=256):
  
#Transformo los input a datasets
.
    dataset=tf.data.Dataset.from_tensor_slices((dict(features), labels))

    #se usa Shuffle para reentrenar el modelo 
    if training:
        dataset=dataset.shuffle(1000).repeat()
    return dataset.batch(batch_size)

my_feature_columns=[]
for key in df1.key():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [ ]:
#construyo un modelo_ DNN de clasificacion con 2 capas ocultas con 30 a 10 nodos ocultos cada una.
classifier=tf.estimator.DNNClassifier(feature_columns=my_feature_columns, hidden_units=[30,10],n_classes=3)